# Template Matching


[Template matching (Wikipedia)](https://en.wikipedia.org/wiki/Template_matching)
[Template Matching (OpenCV doc)](https://docs.opencv.org/master/d4/dc6/tutorial_py_template_matching.html)

* the simplest form of object detection
* scans a larger image for a provided template by sliding the template target image across the larger image
* a choice on comparison method used; these methods are all some sort of correlation-based metric: [TemplateMatchModes](https://docs.opencv.org/master/df/dfb/group__imgproc__object.html#gga3a7850640f1fe1f58fe91a2d7583695dac5babb7dfda59544e3e31ea928f8cb16)